# Alberto Dicembre, Assignment #4

In [1]:
from bayesian_network import BayesianNetwork 
import numpy as np

## Testing the Network

In [2]:
net = BayesianNetwork(values=["True", "False"]) 
net.add_node("A")
net.add_node("B")
net.add_node("C")


In [3]:
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C']), Edges: {}, Tables: {'A': {'Prior': [0.5, 0.5]}, 'B': {'Prior': [0.5, 0.5]}, 'C': {'Prior': [0.5, 0.5]}}


True

In [4]:
net.add_edge("A", ["B"])
net.add_edge("B", ["C"])
print(net)


Nodes: dict_keys(['A', 'B', 'C']), Edges: {'A': ['B'], 'B': ['C']}, Tables: {'A': {'Prior': [0.5, 0.5]}, 'B': {'Prior': [0.5, 0.5]}, 'C': {'Prior': [0.5, 0.5]}}


In [5]:
net.check_valid()

Node B probability table doesn't match parents' number of values
Node C probability table doesn't match parents' number of values


False

#### Checking Topological Sort

In [11]:
print(net.topo_sort())

['D', 'A', 'B', 'C']


In [6]:
net.add_prob_table("A", {"Prior" : [0.9, 0.1]})
net.add_prob_table("B", {"True" : [0.8, 0.2], "False" : [0.2, 0.8]})

#### Trying to add a probability table for a node not in the network

In [7]:
net.add_prob_table("D", {"Prior" : [0.9, 0.1]})

Node not in network


### Now we create a valid network

In [8]:
net.add_node("D")
net.add_edge("D", ["C"])
net.add_prob_table("C", {("True", "True") : [0.95, 0.05],
                          ("True", "False") : [0.7, 0.3],
                            ("False", "True") : [0.7, 0.3],
                            ("False", "False") : [0.1, 0.9]})
net.add_prob_table("D", {"Prior" : [0.9, 0.1]})
                          
print(net)
net.check_valid()

Nodes: dict_keys(['A', 'B', 'C', 'D']), Edges: {'A': ['B'], 'B': ['C'], 'D': ['C']}, Tables: {'A': {'Prior': [0.9, 0.1]}, 'B': {'True': [0.8, 0.2], 'False': [0.2, 0.8]}, 'C': {('True', 'True'): [0.95, 0.05], ('True', 'False'): [0.7, 0.3], ('False', 'True'): [0.7, 0.3], ('False', 'False'): [0.1, 0.9]}, 'D': {'Prior': [0.9, 0.1]}}


True

#### Let's try some single node sampling

In [9]:
probs = net.get_probabilities("A") # remember, we gave A probabilities of 0.9 for True, 0.1 for False
print(probs)
for i in range(20):
    print(net.sample(probs))

[0.9, 0.1]
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
False
True
False


### Now we try Ancestral Sampling

In [10]:
print(net.ancestral_sampling(10))

Topological sort: ['D', 'A', 'B', 'C']


TypeError: unhashable type: 'list'